In [1]:
%load_ext watermark
%watermark

Last updated: 2024-05-19T15:08:48.709201-05:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.13.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer

In [3]:
consumidores = pd.read_csv("C:/Users/Danny/Desktop/Analisis tesis/4 - Pipelines/data/df_consumo_varObjetivo.csv")

In [4]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


Como vimos en un apartado anterior, en este modelo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

In [5]:
consumidores.shape

(10000, 40)

**INCISO**

vamos a modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn, pero seguramente será arreglado en el futuro

In [6]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [7]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria Category Encoders par ael procesamiento de variables categoricas y categoricas ordinales

In [8]:
transformador_numerico = Pipeline([
    ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
    ("escalador_numerico", preprocessing.MinMaxScaler())
])

pipeline_numerico = ColumnTransformer([
    ("transformador_numerico", transformador_numerico, col_numericas),
])

pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

In [9]:
pipeline_numerico

ColumnTransformer(transformers=[('transformador_numerico',
                                 Pipeline(steps=[('imputador_numerico',
                                                  SimpleImputer()),
                                                 ('escalador_numerico',
                                                  MinMaxScaler())]),
                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])

Ahora se procede a crear el pipeline categorico

In [10]:
transformador_categorico = ColumnTransformer([
    ("transformador_categorico", OneHotEncoder(handle_unknown="impute"), col_categoricas)
])

pipeline_categorico = Pipeline([
    ("transformador_categorico", transformador_categorico)
])

pipeline_categorico.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [11]:
pipeline_categorico

Pipeline(steps=[('transformador_categorico',
                 ColumnTransformer(transformers=[('transformador_categorico',
                                                  OneHotEncoder(handle_unknown='impute'),
                                                  ['d_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                   'g_01_familiares_consumen_sp_tipo',
                                                   'd2_03_estado_civil_tipo',
                                                   'g_04_probaria_sp_tipo',
                                                   'd2_05_nivel_educativo_tipo',
                                                   'd_09_deprimido_tipo',
                                                   'd2_01_etnia_tipo',...
                                                   'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                   'vive_madre_hogar_tipo',
                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                   'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                   'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                   'd_01_aporta_dinero_hogar_tipo',
                                                   'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                   'd_10_poco_interes_tipo',
                                                   'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))])

`category_encoders.OrdinalEncoders` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [12]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [13]:
transformador_ordinal = ColumnTransformer([
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales), col_ordinales)
])

pipeline_ordinal = Pipeline([
    ("transformador_ordinal", transformador_ordinal)
])
pipeline_ordinal.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0],
       [2, 0, 2, 0, 0, 0],
       [2, 0, 3, 0, 0, 0],
       [2, 2, 3, 0, 0, 0],
       [1, 0, 3, 0, 0, 0]])

Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [14]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [15]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                ColumnTransformer(transformers=[('transformador_numerico',
                                                                 Pipeline(steps=[('imputador_numerico',
                                                                                  SimpleImputer()),
                                                                                 ('escalador_numerico',
                                                                                  MinMaxScaler())]),
                                                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                               ('variables_ordinales',
                                Pipeline(steps=[('transformador_ordinal',
                                                 ColumnTransfor...
                                                                                   'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                   'vive_madre_hogar_tipo',
                                                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                   'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                   'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                   'd_01_aporta_dinero_hogar_tipo',
                                                                                   'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                   'd_10_poco_interes_tipo',
                                                                                   'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))]))])

In [16]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)
consumidores_procesado.shape

(10000, 145)

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [18]:
estimador = LogisticRegression(max_iter=1000)

Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [19]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

Para el fit se pasan los datos que se obtienen de entrenamiento y para el predict los datos de test

In [20]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))]))])),
                ('estimador', LogisticRegression(max_iter=1000))])

In [21]:
predicciones = pipeline_estimador.predict(X_test)

In [22]:
predicciones

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [23]:
resultados = {}

def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="weighted")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="weighted")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="weighted")

In [24]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [25]:
#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [26]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["rl"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)

In [27]:
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,15.407931,0.002392,0.980400,0.992363
rl_precision,15.224841,0.005154,0.981103,0.992354
rl_exhaustividad,15.157394,0.005220,0.980400,0.992363
rl_f1_score,15.694187,0.005444,0.980343,0.992354


### Optimizacion de hiperparametros

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform, loguniform

In [29]:
param_dist_random = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [1000, 2000, 3000]
}

In [30]:
busqueda_random = RandomizedSearchCV(estimator=estimador, 
                   param_distributions=param_dist_random,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [31]:
pipeline_estimador_op = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random)
])

In [32]:
pipeline_estimador_op.fit(X_train, y_train)

C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
160 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver = _check_solver(self.so

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C3756C8C10>,
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      3000],
                                                         'penalty': ['l1',
                                                                     'l2'],
                                                         'solver': ['newton-cg',
                                                                    'lbfgs',
                                                                    'liblinear',
                                                                    'sag',
                                                                    'saga']},
                                    scoring='accuracy'))])

In [33]:
pipeline_estimador_op.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [34]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

0.9807500000000001
LogisticRegression(C=7.035174574590315, max_iter=1000, penalty='l1',
                   solver='saga')


In [35]:
pipeline_estimador_op_best = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random.best_estimator_)
])

In [36]:
pipeline_estimador_op_best.fit(X_train, y_train)

C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=7.035174574590315, max_iter=1000,
                                    penalty='l1', solver='saga'))])

In [37]:
pipeline_estimador_op_best.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [38]:
estimador_best = pipeline_estimador_op_best.named_steps['estimador']
resultados["rl_best"] = evaluar_modelo(estimador_best, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,15.407931,0.002392,0.980400,0.992363
rl_precision,15.224841,0.005154,0.981103,0.992354
rl_exhaustividad,15.157394,0.005220,0.980400,0.992363
rl_f1_score,15.694187,0.005444,0.980343,0.992354
rl_best_exactitud,171.731887,0.001560,0.985100,0.999102
rl_best_precision,163.520619,0.004098,0.985766,0.999102
rl_best_exhaustividad,162.579675,0.004008,0.985100,0.999100
rl_best_f1_score,162.910436,0.004121,0.985066,0.999100


### Segunda optimizacion

In [39]:
param_dist_random2 = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000, 2000, 5000],
    'dual': [False, True],
    'fit_intercept': [True, False]
}

In [40]:
busqueda_random2 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random2,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [41]:
pipeline_estimador_op2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2)
])

In [42]:
pipeline_estimador_op2.fit(X_train, y_train)

C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
185 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
65 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\linear_model\_logistic.py", line 1227, in fit
    self.coef_, self.intercept_, s

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                ('estimador',
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C375687FA0>,
                                                         'dual': [False, True],
                                                         'fit_intercept': [True,
                                                                           False],
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      5000],
                                                         'penalty': ['l1',
                                                                     'l2'],
                                                         'solver': ['liblinear',
                                                                    'saga']},
                                    scoring='accuracy'))])

In [43]:
pipeline_estimador_op2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [44]:
print(busqueda_random2.best_score_)
print(busqueda_random2.best_estimator_)

0.9825000000000002
LogisticRegression(C=3.3772006179620995, max_iter=1000, penalty='l1',
                   solver='saga')


In [45]:
pipeline_estimador_op_best2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2.best_estimator_)
])

In [46]:
pipeline_estimador_op_best2.fit(X_train, y_train)

C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=3.3772006179620995, max_iter=1000,
                                    penalty='l1', solver='saga'))])

In [47]:
pipeline_estimador_op_best2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [48]:
estimador_best2 = pipeline_estimador_op_best2.named_steps['estimador']
resultados["rl_best_2"] = evaluar_modelo(estimador_best2, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,15.407931,0.002392,0.980400,0.992363
rl_precision,15.224841,0.005154,0.981103,0.992354
rl_exhaustividad,15.157394,0.005220,0.980400,0.992363
rl_f1_score,15.694187,0.005444,0.980343,0.992354
rl_best_exactitud,171.731887,0.001560,0.985100,0.999102
rl_best_precision,163.520619,0.004098,0.985766,0.999102
rl_best_exhaustividad,162.579675,0.004008,0.985100,0.999100
rl_best_f1_score,162.910436,0.004121,0.985066,0.999100
rl_best_2_exactitud,154.750730,0.001679,0.985500,0.998459
rl_best_2_precision,155.797336,0.003741,0.986150,0.998461


### Tercera optimizacion

In [49]:
param_dist_random3 = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1'],
    'solver': ['saga'],
    'max_iter': [1000, 2000, 5000, 7000],
    'dual': [False],
    'fit_intercept': [True, False],
    'class_weight': [None, 'balanced'],
    'tol': [1e-4, 1e-3, 1e-2],
    'l1_ratio': [0.1, 0.5, 0.9]
}

In [50]:
busqueda_random3 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random3,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [51]:
pipeline_estimador_op3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3)
])

In [52]:
pipeline_estimador_op3.fit(X_train, y_train)

C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\linear_model\_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C37512AA70>,
                                                         'class_weight': [None,
                                                                          'balanced'],
                                                         'dual': [False],
                                                         'fit_intercept': [True,
                                                                           False],
                                                         'l1_ratio': [0.1, 0.5,
                                                                      0.9],
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      5000,
                                                                      7000],
                                                         'penalty': ['l1'],
                                                         'solver': ['saga'],
                                                         'tol': [0.0001, 0.001,
                                                                 0.01]},
                                    scoring='accuracy'))])

In [53]:
pipeline_estimador_op3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [54]:
print(busqueda_random3.best_score_)
print(busqueda_random3.best_estimator_)

0.9829999999999999
LogisticRegression(C=2.482705253647152, l1_ratio=0.5, max_iter=7000,
                   penalty='l1', solver='saga')


In [55]:
pipeline_estimador_op_best3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3.best_estimator_)
])

In [56]:
pipeline_estimador_op_best3.fit(X_train, y_train)

C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\linear_model\_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=2.482705253647152, l1_ratio=0.5,
                                    max_iter=7000, penalty='l1',
                                    solver='saga'))])

In [57]:
pipeline_estimador_op_best3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [58]:
estimador_best3 = pipeline_estimador_op_best3.named_steps['estimador']
resultados["rl_best_3"] = evaluar_modelo(estimador_best3, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,15.407931,0.002392,0.980400,0.992363
rl_precision,15.224841,0.005154,0.981103,0.992354
rl_exhaustividad,15.157394,0.005220,0.980400,0.992363
rl_f1_score,15.694187,0.005444,0.980343,0.992354
rl_best_exactitud,171.731887,0.001560,0.985100,0.999102
rl_best_precision,163.520619,0.004098,0.985766,0.999102
rl_best_exhaustividad,162.579675,0.004008,0.985100,0.999100
rl_best_f1_score,162.910436,0.004121,0.985066,0.999100
rl_best_2_exactitud,154.750730,0.001679,0.985500,0.998459
rl_best_2_precision,155.797336,0.003741,0.986150,0.998461


### Cuarta optimizacion

In [61]:
param_dist_random4 = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1'],
    'solver': ['saga'],
    'max_iter': [1000, 2000, 5000, 7000, 9000],
    'dual': [False],
    'fit_intercept': [True, False],
    'class_weight': [None, 'balanced'],
    'tol': [1e-4, 1e-3, 1e-2]
}

In [62]:
busqueda_random4 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random4,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [63]:
pipeline_estimador_op4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4)
])

In [64]:
pipeline_estimador_op4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C3756EAF20>,
                                                         'class_weight': [None,
                                                                          'balanced'],
                                                         'dual': [False],
                                                         'fit_intercept': [True,
                                                                           False],
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      5000,
                                                                      7000,
                                                                      9000],
                                                         'penalty': ['l1'],
                                                         'solver': ['saga'],
                                                         'tol': [0.0001, 0.001,
                                                                 0.01]},
                                    scoring='accuracy'))])

In [65]:
pipeline_estimador_op4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [66]:
print(busqueda_random4.best_score_)
print(busqueda_random4.best_estimator_)

0.9828749999999999
LogisticRegression(C=2.630643757616179, fit_intercept=False, max_iter=9000,
                   penalty='l1', solver='saga')


In [67]:
pipeline_estimador_op_best4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4.best_estimator_)
])

In [68]:
pipeline_estimador_op_best4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=2.630643757616179, fit_intercept=False,
                                    max_iter=9000, penalty='l1',
                                    solver='saga'))])

In [69]:
pipeline_estimador_op_best4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [70]:
estimador_best4 = pipeline_estimador_op_best4.named_steps['estimador']
resultados["rl_best_4"] = evaluar_modelo(estimador_best4, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,15.407931,0.002392,0.980400,0.992363
rl_precision,15.224841,0.005154,0.981103,0.992354
rl_exhaustividad,15.157394,0.005220,0.980400,0.992363
rl_f1_score,15.694187,0.005444,0.980343,0.992354
rl_best_exactitud,171.731887,0.001560,0.985100,0.999102
rl_best_precision,163.520619,0.004098,0.985766,0.999102
rl_best_exhaustividad,162.579675,0.004008,0.985100,0.999100
rl_best_f1_score,162.910436,0.004121,0.985066,0.999100
rl_best_2_exactitud,154.750730,0.001679,0.985500,0.998459
rl_best_2_precision,155.797336,0.003741,0.986150,0.998461


### Exportar con joblib el mejor pipeline obtenido

Tenemos que exportar 3 cosas para poder recrear el Pipeline:

**1. El pipeline entrenado.**

Podemos usar la libreria de scikit-learn `joblib` para guardar nuestro pipeline al disco duro. Joblib es una libreria basada en pickle que sirve para guardar objetos de python al disco duro, pero que está optimizado para scikit-learn.

In [71]:
import joblib

In [72]:
joblib.dump(pipeline_estimador_op_best3, 'pipeline_rl_best.pkl')

['pipeline_rl_best.pkl']

In [78]:
clf = joblib.load('C:/Users/Danny/Desktop/Analisis_Modelo/Alisis Danny/4 - Pipelines/Modelo RL/pipeline_rl_best.pkl')

In [79]:
clf.predict(consumidores.head())

array([0, 0, 0, 0, 1], dtype=int64)

**2. Los nombres de las columnas**

Necesitamos saber las columnas que espera el pipeline para poder saber cómo tenemos que formatear los datos en el futuro de forma que los reconozca.

In [83]:
import json
with open('columnas_consumidores.json', 'w') as fname:
    consumidores_columnas = consumidores.columns.tolist()
    json.dump(consumidores_columnas, fname)

**3. Los tipos de datos de cada variable**

Necesitamos que las variables de las nuevas observaciones estén en el mismo formato que espera el pipeline.

In [84]:
consumidores_dtypes = consumidores.dtypes
consumidores_dtypes = {col: consumidores[col].dtype for col in consumidores.columns}
joblib.dump(consumidores_dtypes, 'dtypes_consumidores.pkl')

['dtypes_consumidores.pkl']